In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import warnings
from collections import Counter
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

In [ ]:
age_train = pd.read_csv('../input/age_train.csv', header=None)
age_test = pd.read_csv('../input/age_test.csv', header=None)
userInfo = pd.read_csv('../input/user_basic_info.csv', header=None)
userBehavior = pd.read_csv('../input/user_behavior_info.csv', header=None)
userAppActived = pd.read_csv('../input/user_app_actived.csv', header=None)
appInfo = pd.read_csv('../input/app_info.csv', header=None)

In [ ]:
userInfo.columns=['uId', 'gender', 'city', 'prodName', 'ramCapacity', 'ramLeftRation', 'romCapacity', 'romLeftRation', 'color', 'fontSize', 'ct', 'carrier', 'os']
age_train.columns=['uId', 'age_group']
age_test.columns=['uId']
userBehavior.columns=['uId', 'bootTimes', 'AFuncTimes', 'BFuncTimes', 'CFuncTimes', 'DFuncTimes', 'EFuncTimes', 'FFuncTimes', 'FFuncSum']
userAppActived.columns=['uId', 'appId']
appInfo.columns=['appId', 'category']

In [ ]:
userAppActived = pd.read_csv('input/user_app_actived.csv',  header=None)
userAppActived.columns = ['uId', 'appId']
idArr = []
appArr = []
l = len(userAppActived.index)
for x in range(l):
    print(x, '/', l)
    tmp = userAppActived['appId'].iloc[x]
    if tmp[0] != 'a':
        continue
    tmp = tmp.split('#')
    for j in range(len(tmp)):
        idArr.append(userAppActived['uId'].iloc[x])
        appArr.append(tmp[j])

result = pd.DataFrame({'uId': idArr, 'appId': appArr})
result.merge(appInfo, on='appId', how='left')
userAppUsed = result.groupby('uId').category.value_counts().unstack().reset_index().fillna(0)

In [ ]:
trainData = age_train.merge(userAppUsed, on='uId', how='left')
print(trainData.isnull().sum())
trainLabel = age_train.age_group
testData = age_test.merge(userAppUsed, on='uId', how='left')
print(testData.isnull().sum())
for column in list(trainData.columns[trainData.isnull().sum() > 0]):
    trainData[column].fillna(0, inplace=True)
for column in list(testData.columns[testData.isnull().sum() > 0]):
    testData[column].fillna(0, inplace=True)

In [ ]:
newData = userInfo[['uId', 'gender','ramCapacity', 'ramLeftRation', 'romCapacity', 'romLeftRation', 'fontSize', 'carrier', 'os']]
print(newData.isnull().sum())
carrierEncoder = LabelEncoder().fit(newData.carrier)
newData['carrier'] = carrierEncoder.transform(newData['carrier'])

In [ ]:
for column in list(newData.columns[newData.isnull().sum() > 0]):
    mean_val = newData[column].mean()
    newData[column].fillna(mean_val, inplace=True)

In [ ]:
trainData = trainData.merge(newData, on='uId', how='left')
testData = testData.merge(newData, on='uId', how='left')

In [ ]:
trainData.drop(['age_group'], axis=1, inplace=True)

In [ ]:
trainData.to_csv('firstVersionTrainData.csv', index=False)
testData.to_csv('firstVersionTestData.csv', inde=False)